In [1]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np

# --- 1. Tải 2 bộ dữ liệu chính ---
# Dùng try-except để bắt lỗi FileNotFoundError một cách tường minh
try:
    df_rankings = pd.read_csv('/kaggle/input/world-university-rankings/cwurData.csv')
    df_coursera = pd.read_csv('/kaggle/input/coursera-course-data/coursera_course_dataset_v2_no_null.csv')
    print("✅ Tải dữ liệu thành công!")
except FileNotFoundError as e:
    print(f"❌ LỖI: Không tìm thấy tệp. Vui lòng kiểm tra lại đường dẫn trong phần 'Data'.")
    print(f"Chi tiết lỗi: {e}")


# --- 2. Chuẩn hóa cột cho bộ dữ liệu Coursera ---
# Đổi tên các cột gốc ('Title', 'Organization', 'Skills') thành các tên chuẩn ('major_name', 'university_name', 'major_description')
df_coursera = df_coursera.rename(columns={
    'Title': 'major_name',
    'Organization': 'university_name',
    'Skills': 'major_description'
})

# Tạo một DataFrame mới, độc lập bằng .copy() để tránh cảnh báo SettingWithCopyWarning
final_courses_df = df_coursera[['major_name', 'university_name', 'major_description']].copy()

# Làm sạch dữ liệu trên DataFrame mới
final_courses_df.dropna(subset=['major_description'], inplace=True)
final_courses_df.drop_duplicates(inplace=True)


# --- 3. Hợp nhất dữ liệu khóa học với dữ liệu xếp hạng ---
# Chuẩn hóa tên trường về chữ thường để khớp (merge)
df_rankings['institution'] = df_rankings['institution'].str.lower()
final_courses_df['university_name'] = final_courses_df['university_name'].str.lower()

# Chỉ lấy các cột cần thiết từ bảng xếp hạng
df_rankings_simple = df_rankings[['institution', 'country', 'score']]

# Hợp nhất hai DataFrame
master_df = pd.merge(
    final_courses_df,
    df_rankings_simple,
    left_on='university_name',
    right_on='institution',
    how='left'
)

# --- 4. Xử lý các giá trị bị thiếu sau khi hợp nhất ---
# Thay vì dùng inplace=True, gán lại giá trị một cách trực tiếp để an toàn hơn
master_df['score'] = master_df['score'].fillna(master_df['score'].mean())
master_df['country'] = master_df['country'].fillna('Unknown')

# Đổi tên cột 'score' thành 'university_score' cho rõ ràng
master_df = master_df.rename(columns={'score': 'university_score'})


# --- 5. Tạo DataFrame cuối cùng ---
# Chọn các cột cuối cùng và reset index để đảm bảo tính nhất quán
final_df = master_df[['major_name', 'major_description', 'university_name', 'country', 'university_score']]
final_df = final_df.reset_index(drop=True)

print("\n🎉 Dữ liệu đã sẵn sàng cho Bước 2!")
print("-----------------------------------------")
print("5 dòng dữ liệu đầu tiên:")
print(final_df.head())
print(f"\nKích thước dữ liệu cuối cùng: {final_df.shape[0]} dòng và {final_df.shape[1]} cột.")

✅ Tải dữ liệu thành công!

🎉 Dữ liệu đã sẵn sàng cho Bước 2!
-----------------------------------------
5 dòng dữ liệu đầu tiên:
                              major_name  \
0                   Google Cybersecurity   
1                  Google Data Analytics   
2             Google Project Management:   
3                       IBM Data Science   
4  Google Digital Marketing & E-commerce   

                                   major_description university_name  country  \
0   Network Security, Python Programming, Linux, ...          google  Unknown   
1   Data Analysis, R Programming, SQL, Business C...          google  Unknown   
2   Project Management, Strategy and Operations, ...          google  Unknown   
3   Python Programming, Data Science, Machine Lea...             ibm  Unknown   
4   Digital Marketing, Marketing, Marketing Manag...          google  Unknown   

   university_score  
0         62.399382  
1         62.399382  
2         62.399382  
3         62.399382  
4         

In [2]:
# Cài đặt thư viện googletrans phiên bản ổn định
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=e841943341b96e0c716e7be46bbd96bd9aeaf5cf6803069dcc65b51f1afe7e06
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [3]:
# Import các thư viện cần thiết, thêm Translator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from googletrans import Translator

# ===================================================================
# PHẦN 1: XÂY DỰNG MÔ HÌNH ML (Không thay đổi)
# ===================================================================
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
major_vectors = vectorizer.fit_transform(final_df['major_description'])
print("Đã tạo xong ma trận vector cho các ngành học.")
print("Kích thước ma trận:", major_vectors.shape)

# Khởi tạo đối tượng Translator để tái sử dụng
translator = Translator()

# ===================================================================
# PHẦN 2: VIẾT CÁC HÀM GỢI Ý (Cập nhật)
# ===================================================================

# HÀM 1: LỌC THEO QUY TẮC (Không thay đổi)
def filter_by_rules(df, min_uni_score=60):
    eligible_majors = df[df['university_score'] >= min_uni_score].copy()
    print(f"\nĐang lọc... Tìm thấy {len(eligible_majors)} ngành học từ các trường có điểm >= {min_uni_score}")
    return eligible_majors

# HÀM 2: XẾP HẠNG THEO SỞ THÍCH (Không thay đổi)
def rank_by_interest(df, user_interest, vectorizer_model, major_vectors_matrix):
    if df.empty:
        return df
    user_vector = vectorizer_model.transform([user_interest])
    filtered_indices = df.index
    similarity_scores = cosine_similarity(user_vector, major_vectors_matrix[filtered_indices])
    df['similarity'] = similarity_scores[0]
    return df.sort_values(by='similarity', ascending=False)


# HÀM 3: HÀM TỔNG HỢP (✅ ĐÃ NÂNG CẤP VỚI TÍNH NĂNG DỊCH)
def get_recommendations(user_score_input, user_interest_input, top_n=5):
    """
    Kết hợp cả hai bước lọc và xếp hạng để đưa ra gợi ý.
    Tự động dịch sở thích từ tiếng Việt sang tiếng Anh.
    """
    # Bước A: Dịch sở thích của người dùng
    print(f"\nSở thích của bạn: '{user_interest_input}'")
    translated_interest = translator.translate(user_interest_input, src='vi', dest='en').text
    print(f"---> Đã dịch sang tiếng Anh: '{translated_interest}'")

    # Bước B: Lọc theo quy tắc cứng (điểm số)
    filtered_majors = filter_by_rules(final_df, min_uni_score=user_score_input)

    # Bước C: Xếp hạng các kết quả đã lọc bằng sở thích đã được dịch
    ranked_recommendations = rank_by_interest(
        filtered_majors,
        translated_interest,  # Sử dụng văn bản đã dịch
        vectorizer,
        major_vectors
    )

    # Trả về top N kết quả tốt nhất
    return ranked_recommendations.head(top_n)

print("\nĐã nâng cấp bộ máy gợi ý với tính năng dịch tự động!")

Đã tạo xong ma trận vector cho các ngành học.
Kích thước ma trận: (1084, 306)

Đã nâng cấp bộ máy gợi ý với tính năng dịch tự động!


In [4]:
# ===================================================================
# HÀM MỚI: ĐA DẠNG HÓA KẾT QUẢ
# ===================================================================
def diversify_recommendations(recommendations_df, top_n=5):
    """
    Từ một danh sách gợi ý, chỉ giữ lại gợi ý tốt nhất (điểm similarity cao nhất)
    của mỗi trường đại học để kết quả không bị trùng lặp.
    """
    # Sắp xếp lại một lần nữa để đảm bảo hàng có điểm cao nhất ở trên cùng
    recommendations_df = recommendations_df.sort_values(by='similarity', ascending=False)
    
    # Xóa các dòng bị trùng lặp 'university_name', chỉ giữ lại dòng ĐẦU TIÊN (có điểm cao nhất)
    diversified_df = recommendations_df.drop_duplicates(subset=['university_name'], keep='first')
    
    return diversified_df.head(top_n)


# ===================================================================
# PHẦN 1: NHẬP THÔNG TIN CỦA BẠN (Không thay đổi)
# ===================================================================
my_score = 70
my_interest = "Mình thích làm việc với dữ liệu, tìm kiếm quy luật, và dùng Python với SQL để xây dựng các mô hình học máy."


# ===================================================================
# PHẦN 2: GỌI AI AGENT VÀ XỬ LÝ KẾT QUẢ (✅ ĐÃ NÂNG CẤP)
# ===================================================================

# Bước A: Lấy một danh sách gợi ý lớn hơn (ví dụ: top 20) để có nhiều lựa chọn
initial_recommendations = get_recommendations(my_score, my_interest, top_n=20)

# Bước B: Dùng hàm mới để lọc và đa dạng hóa kết quả, chỉ lấy top 5 cuối cùng
final_recommendations = diversify_recommendations(initial_recommendations, top_n=5)


# ===================================================================
# PHẦN 3: IN KẾT QUẢ (Không thay đổi)
# ===================================================================
print("\n\n================ KẾT QUẢ GỢI Ý (ĐÃ ĐA DẠNG HÓA) ================")
if final_recommendations.empty:
    print("Rất tiếc, không tìm thấy ngành học nào phù hợp với yêu cầu của bạn.")
else:
    for index, row in final_recommendations.iterrows():
        print(f"🎓 Ngành: {row['major_name']}")
        print(f"🏫 Trường: {row['university_name'].title()} ({row['country']})")
        print(f"✨ Mức độ phù hợp sở thích: {row['similarity']:.2f}")
        print("-" * 30)


Sở thích của bạn: 'Mình thích làm việc với dữ liệu, tìm kiếm quy luật, và dùng Python với SQL để xây dựng các mô hình học máy.'
---> Đã dịch sang tiếng Anh: 'I like working with data, finding patterns, and using Python and SQL to build machine learning models.'

Đang lọc... Tìm thấy 196 ngành học từ các trường có điểm >= 70


================ KẾT QUẢ GỢI Ý (ĐÃ ĐA DẠNG HÓA) ================
🎓 Ngành: Application of AI, InsurTech, and Real Estate Technology
🏫 Trường: University Of Pennsylvania (USA)
✨ Mức độ phù hợp sở thích: 0.44
------------------------------
🎓 Ngành: Data Science Capstone
🏫 Trường: Johns Hopkins University (USA)
✨ Mức độ phù hợp sở thích: 0.43
------------------------------
🎓 Ngành: Fundamentals of Machine Learning for Healthcare
🏫 Trường: Stanford University (USA)
✨ Mức độ phù hợp sở thích: 0.37
------------------------------


In [5]:
import joblib

# Lưu lại mô hình TF-IDF đã được huấn luyện
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

# Lưu lại DataFrame đã được làm sạch cuối cùng
final_df.to_csv('final_majors_data.csv', index=False)

print("✅ Đã lưu thành công 2 tệp: 'tfidf_vectorizer.pkl' và 'final_majors_data.csv'")

✅ Đã lưu thành công 2 tệp: 'tfidf_vectorizer.pkl' và 'final_majors_data.csv'
